# Proceso para pre-procesamiento de audio

In [29]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
import subprocess #se utiliza para ejecutar el comando de demucs
import glob
import shutil

## 1.- En caso de tener el audio en mp3 se debe psar a formato .wav

In [2]:
def convert_mp3_to_wav(mp3_path, wav_path):
    """
    Convierte un archivo MP3 a formato WAV.
    
    :param mp3_path: Ruta del archivo MP3 de entrada
    :param wav_path: Ruta donde se guardará el archivo WAV de salida
    """
    # Cargar el archivo MP3
    audio = AudioSegment.from_mp3(mp3_path)
    
    # Exportar a WAV
    audio.export(wav_path, format="wav")

    print(f"Convertido {mp3_path} a {wav_path}")

## 2.- Se eliminan los silencios de el audio

Hay partes donde el audio tiene pausas largas, en este caso el umbral es que si estas pausas son mayores a 250ms, se va a recortar el audio para unirlos, algunas pausas no se eliminan debido a que es parte de la naturaleza de la voz.

In [19]:

def remove_silence_with_varied_lengths(audio_path, output_dir, silence_thresh=-40, min_silence_lengths=[15,25,50,75,100,200,300,400,500], padding=200):
    """
    Aplica diferentes valores de min_silence_len y guarda los resultados para análisis.
    
    :param audio_path: Ruta al archivo de audio de entrada
    :param output_dir: Directorio donde se guardarán los archivos de salida
    :param silence_thresh: Umbral de silencio en dBFS (decibelios relativos a escala de amplitud de 1)
    :param min_silence_lengths: Lista de valores para min_silence_len a probar
    :param padding: Cantidad de audio que se mantendrá antes y después de cada segmento no silencioso en milisegundos
    """
    # Cargar el archivo de audio
    audio = AudioSegment.from_wav(audio_path)
    
    # Asegúrate de que el directorio de salida existe
    os.makedirs(output_dir, exist_ok=True)

    # Obtener el nombre base del archivo (sin la extensión)
    base_name = os.path.splitext(os.path.basename(audio_path))[0]
    
    for min_silence_len in min_silence_lengths:
        chunks = split_on_silence(audio, 
                                  min_silence_len=min_silence_len, 
                                  silence_thresh=silence_thresh, 
                                  keep_silence=padding)
        
        # Unir los segmentos que no tienen silencio
        processed_audio = AudioSegment.silent(duration=0)
        for chunk in chunks:
            processed_audio += chunk
        
        # Guardar el audio procesado con un nombre que refleje el min_silence_len usado
        output_path = os.path.join(output_dir, f"{base_name}_output_{min_silence_len}ms.wav")
        processed_audio.export(output_path, format="wav")



## 3.-Se extraen la parte vocal de el audio

En esta parte se separa la parte vocal y de audio de fondo, para poder tener un audio mas limpio.

In [27]:
def separate_vocals(input_file, output_dir):
    """
    Separa las voces de la música de fondo y guarda el archivo de voz en el directorio especificado.
    
    :param input_file: Ruta al archivo de audio de entrada (MP3 o WAV)
    :param output_dir: Ruta al directorio donde se guardarán los archivos resultantes
    """

    # Asegúrarse de que la carpeta de salida exista
    os.makedirs(output_dir, exist_ok=True)

    # Obtener el nombre base del archivo sin la extensión
    base_name = os.path.splitext(os.path.basename(input_file))[0]

    # Especificar la carpeta de salida para demucs
    demucs_output_dir = output_dir
    
    # Crear el comando para ejecutar demucs
    print("Extracting vocals...")
    command = f"demucs --two-stems=vocals -o {output_dir} {input_file}"
    subprocess.run(command, shell=True)

    
    print(f"Vocal track extracted and saved to {demucs_output_dir}")


## 4.- Separacion del audio en segmentos de audio <= 15s, basado en los momentos del audio donde se encuentran pausas o silencios.

In [5]:
def split_audio_on_pauses(audio_path, output_dir, min_segment_len=10000, max_segment_len=15000, silence_thresh=-40, silence_padding=200):
    """
    Divide un archivo de audio en segmentos de 10 a 15 segundos, recortando en pausas de silencio.

    :param audio_path: Ruta al archivo de audio de entrada (WAV o MP3)
    :param output_dir: Directorio donde se guardarán los segmentos de salida
    :param min_segment_len: Longitud mínima de los segmentos en milisegundos
    :param max_segment_len: Longitud máxima de los segmentos en milisegundos
    :param silence_thresh: Umbral de silencio en dBFS (decibelios relativos a escala de amplitud de 1)
    :param silence_padding: Cantidad de audio que se mantendrá antes y después de cada pausa en milisegundos
    """
    # Cargar el archivo de audio
    audio = AudioSegment.from_file(audio_path)

    # Dividir el audio basado en pausas
    chunks = split_on_silence(audio, 
                              min_silence_len=silence_padding, 
                              silence_thresh=silence_thresh, 
                              keep_silence=True)
    
    # Crear directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    current_segment = AudioSegment.silent(duration=0)
    segment_count = 0

    for chunk in chunks:
        current_segment += chunk
        
        # Si la longitud del segmento actual está entre los límites especificados, guárdalo
        if min_segment_len <= len(current_segment) <= max_segment_len:
            segment_count += 1
            segment_name = f"segment_{segment_count}.wav"
            current_segment.export(f"{output_dir}/{segment_name}", format="wav")
            current_segment = AudioSegment.silent(duration=0)
        elif len(current_segment) > max_segment_len:
            segment_count += 1
            segment_name = f"segment_{segment_count}.wav"
            current_segment.export(f"{output_dir}/{segment_name}", format="wav")
            current_segment = AudioSegment.silent(duration=0) + chunk
    
    # Exportar el último segmento si no está vacío
    if len(current_segment) > 0:
        segment_count += 1
        segment_name = f"segment_{segment_count}.wav"
        current_segment.export(f"{output_dir}/{segment_name}", format="wav")


## Mover archivos de vocales a una carpeta

In [34]:
def move_vocals_files(source_base_path, destination_dir, start, end):
    """
    Itera sobre un rango de números 'i' para mover archivos vocals.wav desde carpetas con nombres basados en 'i'.
    
    :param source_base_path: Ruta base de las carpetas que contienen los archivos vocals.wav
    :param destination_dir: Directorio donde se moverán los archivos vocals.wav
    :param start: Valor inicial de 'i' para el iterador
    :param end: Valor final de 'i' para el iterador
    """
    # Asegúrate de que la carpeta de destino existe
    os.makedirs(destination_dir, exist_ok=True)
    
    for i in range(start, end + 1):
        # Construir la ruta de la carpeta de origen basada en 'i'
        folder_name = f"segment_{i}_output_250ms"
        folder_path = os.path.join(source_base_path, folder_name)
        print(folder_path)
        # Ruta completa al archivo vocals.wav en la carpeta actual
        vocals_file = os.path.join(folder_path, "vocals.wav")
        
        if os.path.exists(vocals_file):
            # Generar un nombre único basado en el nombre de la carpeta
            new_file_name = f"{folder_name}_vocals.wav"
            
            # Ruta completa al archivo destino
            dest_file = os.path.join(destination_dir, new_file_name)
            
            # Mover el archivo
            shutil.move(vocals_file, dest_file)
            print(f"Moved {vocals_file} to {dest_file}")
        else:
            print(f"File not found: {vocals_file}")



## Borrar folder y archivos en el

In [39]:
def delete_folders(folders_to_delete):
    """
    Elimina una lista de carpetas y todo su contenido.
    
    :param folders_to_delete: Lista de rutas de carpetas que deseas eliminar
    """
    for folder_path in folders_to_delete:
        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
            print(f"Deleted folder and its contents: {folder_path}")
        else:
            print(f"Folder not found: {folder_path}")


## Ejemplo de uso

In [ ]:
#COnvertimos el archivo mp3 a wav
convert_mp3_to_wav("input.mp3", "output.wav")

In [22]:
#Primro si el audio es largo, debemos segmentarlo en partes de 10 a 15 segundos, considerando posibles pausas de silencio, ya que si tiene
#ruido de fondo, solo va a segmentar donde considere tener pausas, esto es para que el procesamiento de extracción de voces sea más preciso y tarde menos.

split_audio_on_pauses("output.wav", "output_segments_no_preprocessing", min_segment_len=10000, max_segment_len=15000, silence_thresh=-30, silence_padding=200)

In [23]:
#Una vez se tiene la primera segmentacion, se procede a remover supuestos silencios de la misma manera que antes, pero a cada segmento
folder_path = "output_segments_no_preprocessing"

# Encuentra todos los archivos .mp3 y .wav en la carpeta de entrada
audio_files = glob.glob(os.path.join(folder_path, '*.mp3')) + glob.glob(os.path.join(folder_path, '*.wav'))
print(audio_files)
    
    # Itera sobre cada archivo y aplica la función de eliminación de silencios
for file in audio_files:
    print(file)
    #Eliminamos algunos supuestos silencios del audio primero
    remove_silence_with_varied_lengths(file, "output_processed_silence", min_silence_lengths=[250],silence_thresh=-40, padding=150)

['output_segments_no_preprocessing\\segment_1.wav', 'output_segments_no_preprocessing\\segment_2.wav', 'output_segments_no_preprocessing\\segment_3.wav', 'output_segments_no_preprocessing\\segment_4.wav', 'output_segments_no_preprocessing\\segment_5.wav', 'output_segments_no_preprocessing\\segment_6.wav']
output_segments_no_preprocessing\segment_1.wav
output_segments_no_preprocessing\segment_2.wav
output_segments_no_preprocessing\segment_3.wav
output_segments_no_preprocessing\segment_4.wav
output_segments_no_preprocessing\segment_5.wav
output_segments_no_preprocessing\segment_6.wav


In [28]:
#Despues podemos aplicar la separación de voces y sonido de fondo, 
# ya que el procesamiento sera mas preciso, para no fallar en extraer las vocales

folder_path_processeced_silenice = "output_processed_silence"

# Encuentra todos los archivos .mp3 y .wav en la carpeta de entrada
audio_files = glob.glob(os.path.join(folder_path_processeced_silenice, '*.mp3')) + glob.glob(os.path.join(folder_path_processeced_silenice, '*.wav'))
print(audio_files)
    
    # Itera sobre cada archivo y aplica la función de eliminación de silencios
for file in audio_files:
    print(file)
    separate_vocals(file,"preprocess_audios_end")

['output_processed_silence\\segment_1_output_250ms.wav', 'output_processed_silence\\segment_2_output_250ms.wav', 'output_processed_silence\\segment_3_output_250ms.wav', 'output_processed_silence\\segment_4_output_250ms.wav', 'output_processed_silence\\segment_5_output_250ms.wav', 'output_processed_silence\\segment_6_output_250ms.wav']
output_processed_silence\segment_1_output_250ms.wav
Extracting vocals...
Vocal track extracted and saved to preprocess_audios_end
output_processed_silence\segment_2_output_250ms.wav
Extracting vocals...
Vocal track extracted and saved to preprocess_audios_end
output_processed_silence\segment_3_output_250ms.wav
Extracting vocals...
Vocal track extracted and saved to preprocess_audios_end
output_processed_silence\segment_4_output_250ms.wav
Extracting vocals...
Vocal track extracted and saved to preprocess_audios_end
output_processed_silence\segment_5_output_250ms.wav
Extracting vocals...
Vocal track extracted and saved to preprocess_audios_end
output_proces

In [33]:
# Ejemplo de uso
source_base_path = "D:\\preprocesamiento_audios\\preprocess_audios_end\\htdemucs"
destination_dir = "D:\\preprocesamiento_audios\\vocals"
start = 1  # Valor inicial de 'i'
end = 6  # Valor final de 'i'

move_vocals_files(source_base_path, destination_dir, start, end)


D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_1_output_250ms
Moved D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_1_output_250ms\vocals.wav to D:\preprocesamiento_audios\vocals\segment_1_output_250ms_vocals.wav
D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_2_output_250ms
Moved D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_2_output_250ms\vocals.wav to D:\preprocesamiento_audios\vocals\segment_2_output_250ms_vocals.wav
D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_3_output_250ms
Moved D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_3_output_250ms\vocals.wav to D:\preprocesamiento_audios\vocals\segment_3_output_250ms_vocals.wav
D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_4_output_250ms
Moved D:\preprocesamiento_audios\preprocess_audios_end\htdemucs\segment_4_output_250ms\vocals.wav to D:\preprocesamiento_audios\vocals\segment_4_output_250ms_vocals.wav


In [37]:
#Una vez extraidas las vocales, podemos eliminar los silencios de forma mas precisa
folder_path = destination_dir

# Encuentra todos los archivos .mp3 y .wav en la carpeta de entrada
audio_files = glob.glob(os.path.join(folder_path, '*.mp3')) + glob.glob(os.path.join(folder_path, '*.wav'))
print(audio_files)
    
# Itera sobre cada archivo y aplica la función de eliminación de silencios
for file in audio_files:
    print(file)
    #Eliminamos algunos supuestos silencios del audio primero
    remove_silence_with_varied_lengths(file, "vocals_end", min_silence_lengths=[250],silence_thresh=-50, padding=250)

['D:\\preprocesamiento_audios\\vocals\\segment_1_output_250ms_vocals.wav', 'D:\\preprocesamiento_audios\\vocals\\segment_2_output_250ms_vocals.wav', 'D:\\preprocesamiento_audios\\vocals\\segment_3_output_250ms_vocals.wav', 'D:\\preprocesamiento_audios\\vocals\\segment_4_output_250ms_vocals.wav', 'D:\\preprocesamiento_audios\\vocals\\segment_5_output_250ms_vocals.wav', 'D:\\preprocesamiento_audios\\vocals\\segment_6_output_250ms_vocals.wav']
D:\preprocesamiento_audios\vocals\segment_1_output_250ms_vocals.wav
D:\preprocesamiento_audios\vocals\segment_2_output_250ms_vocals.wav
D:\preprocesamiento_audios\vocals\segment_3_output_250ms_vocals.wav
D:\preprocesamiento_audios\vocals\segment_4_output_250ms_vocals.wav
D:\preprocesamiento_audios\vocals\segment_5_output_250ms_vocals.wav
D:\preprocesamiento_audios\vocals\segment_6_output_250ms_vocals.wav


Finalmente si todo esta correcto, se eliminan las carpetas y archivos del pre.procesamiento inicial, para no usar memoria o archivos que no sirven.

In [40]:
#Folders a eliminar (OJO CON LOS FOLDERS, VERIFICA SI SON LOS CORRECTOS)
folders_to_delete = [
    "D:\\preprocesamiento_audios\\preprocess_audios_end",
    "D:\\preprocesamiento_audios\\output_processed_silence",
    "D:\\preprocesamiento_audios\\output_segments_no_preprocessing",
    "D:\\preprocesamiento_audios\\vocals"
]

delete_folders(folders_to_delete)


Deleted folder and its contents: D:\preprocesamiento_audios\preprocess_audios_end
Deleted folder and its contents: D:\preprocesamiento_audios\output_processed_silence
Deleted folder and its contents: D:\preprocesamiento_audios\output_segments_no_preprocessing
Deleted folder and its contents: D:\preprocesamiento_audios\vocals
